In [140]:
import librosa
import os
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
import numpy as np
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
#from torchsummaryX import summary
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
import csv
import torch.optim as optim
from tqdm import tqdm
#import feature_extraction.py

In [5]:
with open('./data/processed_data.pkl', 'rb') as f:
    data = pickle.load(f)

In [27]:
data.head()

,Clip_ID,Extraversion,Agreeableness,Conscientiousness,Neuroticism,Openness,Speaker_ID,Gender,Status,features
0,cut_feb1601-guest-0-2,1.727273,-1.363636,1.545455,1.545455,0.727273,92,M,J,"[[0.00027620237, 0.0016822875, 0.001016233, 0...."
1,cut_feb1601-guest-2-10,0.727273,1.181818,1.909091,-1.727273,0.909091,183,M,J,"[[0.38929665, 0.1189508, 0.0048722113, 0.00183..."
2,cut_feb1601-human-6,2.363636,-0.636364,1.363636,-0.545455,0.363636,41,M,G,"[[0.7792231, 0.29407483, 0.03487174, 0.0134306..."
3,cut_feb1602-guest-1-5,1.272727,0.636364,1.818182,-0.454545,0.000000,3,M,J,"[[0.10963277, 0.09769615, 0.021039093, 0.02427..."
4,cut_feb1602-guest-3-17,1.363636,-0.909091,2.181818,0.909091,0.636364,123,M,J,"[[0.077622935, 0.14913887, 0.041750234, 0.0176..."


In [21]:
#Split dataframe into train, validation, and test sets
train, val, test = np.split(data.sample(frac=1, random_state=101),
[int(.7*len(data)), int(.85*len(data))])

In [14]:
type(data['features'].iloc[0])

numpy.ndarray

In [34]:
train['features'].iloc[5].shape

(128, 431)

In [115]:
class LibriSamples(torch.utils.data.Dataset):
    #Partition: the dataframe
    def __init__(self, dataframe, test_order=None, trait=None):
        #self.train=train

        #The personality trait for this run through
        self.trait = trait
        self.dataframe = dataframe
        #self.partition = partition

    def __len__(self):
        return len(self.dataframe.index)
    

    def __getitem__(self, ind):
        #X = np.load(self.dataframe['features'].iloc[ind])
        X = self.dataframe['features'].iloc[ind]
        #Must transpose X so the the first value is length of audio and second value is number of features
        X = X.transpose()
        #if self.train:
        #Y = np.array(self.dataframe[self.trait].iloc[ind])
        Y = torch.tensor([self.dataframe[self.trait].iloc[ind]])
        #if self.train:
        return X, Y
        #else:
        #return X


    def collate_fn(batch):
        batch_x = [torch.from_numpy(x) for x,y in batch]
        batch_y = [y for x,y in batch]
        batch_x_pad = pad_sequence(batch_x, batch_first=True)

        lengths_x = [x.shape[0] for x in batch_x]

        #The y's should only have a length of 1, but we'll pad it anyway

        batch_y_pad = pad_sequence(batch_y, batch_first=True)
        #lengths_y = [y.shape[0] for y in batch_y]
        lengths_y = [y.shape[0] for y in batch_y]

        return batch_x_pad, batch_y_pad, torch.tensor(lengths_x), torch.tensor(lengths_y)


In [164]:
train_data = LibriSamples(train, trait='Extraversion')
val_data = LibriSamples(val, trait='Extraversion')
test_data = LibriSamples(test, trait='Extraversion')

In [165]:
batch_size = 64
lr = 2e-3
epochs=2

In [173]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, shuffle=True, collate_fn = LibriSamples.collate_fn)
val_loader = torch.utils.data.DataLoader(val_data, batch_size = batch_size, shuffle=False, collate_fn = LibriSamples.collate_fn) 
test_loader = torch.utils.data.DataLoader(test_data, batch_size = batch_size, shuffle=False, collate_fn = LibriSamples.collate_fn) 

print("Batch size: ", batch_size)
print("Train dataset samples = {}, batches = {}".format(train_data.__len__(), len(train_loader)))
print("Val dataset samples = {}, batches = {}".format(val_data.__len__(), len(val_loader)))
print("Test dataset samples = {}, batches = {}".format(test_data.__len__(), len(test_loader)))

Batch size:  64
Train dataset samples = 448, batches = 7
Val dataset samples = 96, batches = 2
Test dataset samples = 96, batches = 2


In [174]:
for data in val_loader:
    x, y, lx, ly = data
    print(x.shape, y.shape, lx.shape, ly.shape)
    break

torch.Size([64, 431, 128]) torch.Size([64, 1]) torch.Size([64]) torch.Size([64])


In [175]:
import pdb

In [176]:
class Network(nn.Module):

    def __init__(self): # You can add any extra arguments as you wish

        super(Network, self).__init__()
        
        self.lstm = nn.LSTM(input_size=431, hidden_size = 256, num_layers=1, dropout=.1, bidirectional=True, batch_first=True)

        self.classification = nn.Sequential(nn.ReLU(),
                                            nn.Linear(512,1))
            

    def forward(self, x, x_length): 
        x = torch.transpose(x, 1, 2)
        packed_input = pack_padded_sequence(x,x_length, batch_first = True, enforce_sorted=False)

        
        #packed_input = pack_padded_sequence(x,length.clamp(max=max_expected_len), batch_first = True, enforce_sorted=False)
        #pdb.set_trace()
        out1, (out2, out3) =  self.lstm(packed_input) 
        # As you may see from the LSTM docs, LSTM returns 3 vectors. Which one do you need to pass to the next function?
        out, lengths  = pad_packed_sequence(out1, batch_first=True) 
        #pdb.set_trace()
        #pdb.set_trace()
        out = self.classification(out) 

        #Is dimension 2 or 3? Or something else?
   
        #m = n.LogSoftmax(dim=2) # Optional: Do log softmax on the output. Which dimension?
        out = F.log_softmax(out,dim=2)


        return out, lengths 

#model = Network().to(device)

In [177]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = Network().to(device)
print(model)

Network(
  (lstm): LSTM(431, 256, batch_first=True, dropout=0.1, bidirectional=True)
  (classification): Sequential(
    (0): ReLU()
    (1): Linear(in_features=512, out_features=1, bias=True)
  )
)


C:\Users\cgwed\Anaconda3\lib\site-packages\torch\nn\modules\rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [178]:
criterion = nn.L1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr = lr)
scaler = torch.cuda.amp.GradScaler()


C:\Users\cgwed\Anaconda3\lib\site-packages\torch\cuda\amp\grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")


In [179]:
for epoch in range(epochs):
    batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train') 
    model.train()
    total_loss = 0

    for i, data in enumerate(train_loader):
        optimizer.zero_grad()

        x = data[0].to(device)
        y = data[1]
        x_lengths = data[2]
        with torch.cuda.amp.autocast():
            outputs = model(x, x_lengths)
            loss = criterion(outputs, y)

    total_loss += loss
    batch_bar.set_postfix(
        loss="{:.04f}".format(float(total_loss/(i+1)))
    )

    scaler.scale(loss).backward()
    scaler.step(optimizer)
    scaler.update()

    batch_bar.update()

C:\Users\cgwed\Anaconda3\lib\site-packages\torch\autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


RuntimeError: start (8192) + length (64) exceeds dimension size (8192).

In [ ]:
len(train_loader)

7